In [55]:
import os
import pandas as pd
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from datetime import datetime
import scipy.signal as signal
from scipy.integrate import odeint
from scipy.optimize import minimize, rosen, rosen_der

In [34]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))


In [35]:
data={}
for file in csv_files:
    
    p=Path(file)
    filename=p.with_suffix('').stem
    finalfilename=filename.replace("temperature_", "")
    data[finalfilename]=pd.read_csv(p)
    bad_data=data[finalfilename]



In [340]:
data['heating_system'].tail(5)

,time,water_pressure,water_temperature
102681,2021-05-24T16:55:03Z,1.2,26
102682,2021-05-24T17:00:04Z,1.5,30
102683,2021-05-24T17:05:03Z,1.2,29
102684,2021-05-24T17:10:03Z,1.2,29
102685,2021-05-24T17:15:04Z,1.5,29


In [37]:
data.keys()


dict_keys(['pv_production_load', 'bathroom', 'bedroom_1', 'bedroom_2', 'bedroom_3', 'diningroom', 'heating_system', 'kitchen', 'livingroom', 'outside'])

In [338]:
data['livingroom'].tail()

,time,current_value,setpoint
0,2020-05-24T17:15:03Z,21.8,21.0
1,2020-05-24T17:20:03Z,21.8,21.0
2,2020-05-24T17:25:03Z,21.8,21.0
3,2020-05-24T17:30:04Z,21.8,21.0
4,2020-05-24T17:35:04Z,22.1,21.0


In [39]:
npdata={}
for key in data.keys():
    npdata[key]=data[key].to_numpy()

In [347]:
npdata['average'] = np.empty_like(npdata['kitchen'])


array([[None, None, None],
       [None, None, None],
       [None, None, None],
       ...,
       [None, None, None],
       [None, None, None],
       [None, None, None]], dtype=object)

In [41]:
npdata.keys()

dict_keys(['pv_production_load', 'bathroom', 'bedroom_1', 'bedroom_2', 'bedroom_3', 'diningroom', 'heating_system', 'kitchen', 'livingroom', 'outside', 'average'])

In [42]:
area={}
area['bathroom']=54*(514-232)
area['annex_bathroom']=(1388-1283)*(514-230)
area['kitchen']=(911-640)*(511-234)
area['bedroom_1']=(1277-1013)*(512-235)
area['bedroom_2']=(1165-1014)*(717-618)+(1229-1003)*(816-716)+(1329-1012)*(893-819)
area['bedroom_3']=(1551-1341)*(896-612)
area['bathroom_2']=(1326-1238)*(805-716)
area['annex_bathroom_2']=(1330-1175)*(708-614)
area['diningroom']=(634-374)*(512-234)
area['stairs']=(914-371)*(605-519)+(1385-1010)*(606-519)
area['storage']=(1549-1396)+(606-520)
area['diningroom']=(676-371)*(895-616)
area['annex_livingroom']=(914-685)*(893-614)

In [375]:


for i in range(0,npdata['average'].shape[0]):
    stamp = npdata['kitchen'][i][0]
    temperature = np.average([npdata['bathroom'][i][1],
                             npdata['bedroom_1'][i][1],
                             npdata['bedroom_2'][i][1],
                             npdata['bedroom_3'][i][1],
                             npdata['diningroom'][i][1],
                             npdata['kitchen'][i][1],
                             npdata['livingroom'][i][1]])
    setpoint =  np.average([npdata['bathroom'][i][2],
                             npdata['bedroom_1'][i][2],
                             npdata['bedroom_2'][i][2],
                             npdata['bedroom_3'][i][2],
                             npdata['diningroom'][i][2],
                             npdata['kitchen'][i][2],
                             npdata['livingroom'][i][2]])
    npdata['average'][i] = np.array([0,temperature,setpoint])

In [376]:
npdata['average'][0]

array([0.0, 21.97142857142857, 17.714285714285715], dtype=object)

In [208]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
solardata = moving_average(npdata['pv_production_load'][:,1],12)
test

array([0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0], dtype=object)

In [246]:
def controlfunction(Trad, setT, Fmin, Fmax):
    if Trad < (setT -2):
        return Fmax
    elif Trad > (setT+2):
        return Fmin
    else:
        return Fmax - (Fmax-Fmin)*(Trad-setT)/4

In [247]:
npdata['heating_system'][:,1]

array([1.3, 1.6, 1.3, ..., 1.2, 1.2, 1.5], dtype=object)

In [386]:
#params0=[C_eau,C_zone,R_rad2zone, R_zone2ext,solarparameter,controlfunction_parameter1,controlfunction_parameter2]
#externe=[T_ext,setT,controlfunction,solardata,T_boiler]

# function that returns dstate/dt

#probleme à résoudre : le modèle dépend de la température extérieure qui change

externe = [npdata['outside'][:,1],npdata['average'][:,2],
           controlfunction,solardata, npdata['heating_system'][:,2]]
solardata = npdata['pv_production_load'][:,1]

def model0(X,t,params0):
    
    
    
    T_rad, T_zone = X
    
    dT_zone = ((externe[0][round(t)]-T_zone)/params0[3]
               + (T_rad-externe[0][round(t)])/params0[2]
               + params0[4]*externe[3][round(t)])/params0[1]
    
    dT_rad = externe[2](T_rad,float(externe[1][round(t)]),params0[5],params0[6]) * (externe[4][round(t)]-T_rad)/params0[0]
    
    
    
    return [dT_rad, dT_zone]
    

    

In [387]:
externe[1][round(0.5)]

17.714285714285715

In [388]:
controlfunction(10,float(externe[1][round(0.5)]),9,15)

15

In [389]:
params0=[500,100,100,100,100,1,1]
y0=[npdata['heating_system'][0][1],npdata['heating_system'][0][1]]

nbligne=len(npdata['bathroom'])

ret=odeint(model0, y0, t = np.linspace(-0.5, 10 -0.5, 10), args=(params0,))

nbligne

102686

In [390]:
np.sum((ret[:,0]-ret[:,1])**2)

0.7592847208946516

In [406]:
def residual(ps):
    #cette fonction doit donner une valeur en fonction des paramètres
    
    sim =  odeint(model0, y0, t = np.linspace(10-0.5, nbligne-0.5-10, nbligne-20), args=(ps,))
    return np.sum((sim[:,1]-npdata['average'][10:-10,1])**2)

In [407]:
npdata['average'][10:-10,1][0]

22.085714285714285

In [408]:
odeint(model0, y0, t = np.linspace(-0.5, 10 -0.5, 10), args=(params0,))

array([[1.3       , 1.3       ],
       [1.35038845, 1.3000028 ],
       [1.40110939, 1.3000112 ],
       [1.45349315, 1.30002534],
       [1.50576062, 1.3000453 ],
       [1.55791208, 1.30007105],
       [1.60994777, 1.30010259],
       [1.66186795, 1.3001399 ],
       [1.71367288, 1.30018297],
       [1.76536282, 1.30023178]])

In [409]:
residual(params0)

7.710842038850871e+17

In [ ]:
result = minimize(residual,params0)